In [3]:
### adding libraries for store vectors & metadata
import faiss
import numpy as np
import pandas as pd
import pickle
from sentence_transformers import SentenceTransformer, util
from tqdm import tqdm
import tqdm as notebook_tqdm
import time

c:\Users\My-PC\miniconda3\envs\tf310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:

### loading the data
df_pt = pd.read_csv("../data/potential-talents.csv")
df_pt.head()

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [5]:
scores = [0.2, 0.9, 0.1, 0.8]
first_scores = scores[0]
print(first_scores)

0.2


In [6]:
sentences = df_pt['job_title'].tolist()


In [7]:
len(sentences)

104

In [8]:
sentences = [word for word in list(sentences) if isinstance(word, str)]  # filtering out non-string entries
len(sentences)

104

In [9]:
sentences_df = pd.DataFrame({'sentences': sentences})

In [10]:
data  = pd.concat([sentences_df, df_pt], axis=1)
data.head()

,sentences,id,job_title,location,connection,fit
0,2019 C.T. Bauer College of Business Graduate (...,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,Native English Teacher at EPIK (English Progra...,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,Aspiring Human Resources Professional,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,People Development Coordinator at Ryan,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,Advisory Board Member at Celal Bayar University,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN


In [11]:
df_pt.shape

(104, 5)

In [12]:
data.shape

(104, 6)

In [13]:
from sentence_transformers import SentenceTransformer, util
model_2 =SentenceTransformer('all-MiniLM-L6-v2')
sentence_embeddings_2 = model_2.encode(sentences,convert_to_tensor=True,show_progress_bar=True)
sentence_embeddings_2.shape

Batches: 100%|██████████| 4/4 [00:00<00:00,  6.35it/s]


torch.Size([104, 384])

In [14]:
import numpy as np
np.save('../data/sentence_embeddings_2.npy', sentence_embeddings_2.cpu().numpy())

In [15]:
sentence_embeddings_2 = np.load('../data/sentence_embeddings_2.npy')

In [16]:
import faiss

In [17]:
d = sentence_embeddings_2.shape[1]  # dimension
index = faiss.IndexFlatL2(d)  # build the index 
print(index.is_trained)

True


In [18]:
index.add(sentence_embeddings_2)  # add vectors to the index
print(index.ntotal)

104


In [48]:
k =10
xq = model_2.encode(["Aspiring Human Resources Specialist"])

In [45]:
D, I = index.search(np.array(xq), k)  # search
print(I)

[[ 5 23 35 48 59  2 16 20 32 45]]


In [46]:
print(I[0])

[ 5 23 35 48 59  2 16 20 32 45]


In [35]:
data['sentences'].iloc[[5, 23, 35, 48, 59,  2, 16, 20, 32, 45]]

5       Aspiring Human Resources Specialist
23      Aspiring Human Resources Specialist
35      Aspiring Human Resources Specialist
48      Aspiring Human Resources Specialist
59      Aspiring Human Resources Specialist
2     Aspiring Human Resources Professional
16    Aspiring Human Resources Professional
20    Aspiring Human Resources Professional
32    Aspiring Human Resources Professional
45    Aspiring Human Resources Professional
Name: sentences, dtype: object

In [47]:
data.iloc[I[0]]

,sentences,id,job_title,location,connection,fit
5,Aspiring Human Resources Specialist,6,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
23,Aspiring Human Resources Specialist,24,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
35,Aspiring Human Resources Specialist,36,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
48,Aspiring Human Resources Specialist,49,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
59,Aspiring Human Resources Specialist,60,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
2,Aspiring Human Resources Professional,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
16,Aspiring Human Resources Professional,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
20,Aspiring Human Resources Professional,21,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
32,Aspiring Human Resources Professional,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
45,Aspiring Human Resources Professional,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN


In [37]:
vecs = np.zeros([k, d])
for i,val in enumerate(I[0].tolist()):
    vecs[i, :] = index.reconstruct(val)

In [38]:
print(vecs.shape)

(10, 384)


In [52]:
nlist = 10
quantizer = faiss.IndexFlatL2(d)  # the other index
index2 = faiss.IndexIVFFlat(quantizer, d, nlist, faiss.METRIC_L2)

In [53]:
index2.is_trained

False

In [54]:
index2.train(sentence_embeddings_2)
print(index2.is_trained)
index2.add(sentence_embeddings_2)

True


In [55]:
print(index2.ntotal)

104


In [56]:
D, I = index2.search(np.array(xq), k)  # search
print(I)

[[ 5 23 35 48 59  2 16 20 32 45]]


In [57]:
data.iloc[I[0]]

,sentences,id,job_title,location,connection,fit
5,Aspiring Human Resources Specialist,6,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
23,Aspiring Human Resources Specialist,24,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
35,Aspiring Human Resources Specialist,36,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
48,Aspiring Human Resources Specialist,49,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
59,Aspiring Human Resources Specialist,60,Aspiring Human Resources Specialist,Greater New York City Area,1,NaN
2,Aspiring Human Resources Professional,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
16,Aspiring Human Resources Professional,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
20,Aspiring Human Resources Professional,21,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
32,Aspiring Human Resources Professional,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
45,Aspiring Human Resources Professional,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
